#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [2]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_1_6')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [3]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd/bin'

# Construct the command
#command = [
#    './bin/neo4j-admin', 'database', 'import', 'full',
#    '--nodes=import/nodes.csv',
#    '--relationships=import/relationships.csv', 'neo4j'
#]

command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd\import\nodes.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
  Free machine memory: 21.96GiB
  Max heap memory : 910.5MiB
  Max worker threads: 8
  Configured max memory: 19.05GiB
  High parallel IO: true

Cypher type normalization is enabled (disable with --normalize-types=false):
  Property type of 'Weight' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-dc7b2527-0ef7-41cd-93b4-c56885df2bfd\import\relationships.csv

Import starting 2024-03-09 19:53:12.874+0000
  Estimated number of nodes: 3.05 k
  Estimated numbe

Run and connect to the Neo4j Database

In [4]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [5]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [6]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [7]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [8]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [9]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [10]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [11]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [12]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [13]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [14]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

In [15]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, label_propagation_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on='name', how='left')

In [16]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain
0,210519-3-01,1.522488,18235.0,0.029951,0.659144,1,2548,1160
1,210217-2-11,1.504415,18017.0,0.029593,0.652600,1,2548,1160
2,210217-2-05,1.471457,17589.0,0.028890,0.640493,1,2548,1160
3,210217-2-04,1.471432,17591.0,0.028893,0.640519,1,2548,1160
4,210217-2-18,1.469541,17574.0,0.028865,0.639833,1,2548,1160
...,...,...,...,...,...,...,...,...
3040,210519-2-37,0.417426,3510.0,0.005765,0.248813,1,2548,1160
3041,210114-1-17,0.399481,3243.0,0.005327,0.241883,1,1,2299
3042,210304-2-20,0.397664,3213.0,0.005277,0.241136,1,1,2299
3043,210211-2-27,0.386780,3080.0,0.005059,0.237206,1,1,2299


Delete the projection

In [17]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [18]:
close_driver()

In [19]:
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain
0,210519-3-01,1.522488,18235.0,0.029951,0.659144,1,2548,1160
1,210217-2-11,1.504415,18017.0,0.029593,0.652600,1,2548,1160
2,210217-2-05,1.471457,17589.0,0.028890,0.640493,1,2548,1160
3,210217-2-04,1.471432,17591.0,0.028893,0.640519,1,2548,1160
4,210217-2-18,1.469541,17574.0,0.028865,0.639833,1,2548,1160
...,...,...,...,...,...,...,...,...
3040,210519-2-37,0.417426,3510.0,0.005765,0.248813,1,2548,1160
3041,210114-1-17,0.399481,3243.0,0.005327,0.241883,1,1,2299
3042,210304-2-20,0.397664,3213.0,0.005277,0.241136,1,1,2299
3043,210211-2-27,0.386780,3080.0,0.005059,0.237206,1,1,2299


In [20]:
status_df = pd.read_csv('../../data/graph_1_6/nodes.csv')

In [21]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [22]:
graph_df = pd.merge(df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
graph_df.drop(columns=["SpecID:ID"], inplace=True)

In [23]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [24]:
graph_df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain,Status
0,210519-3-01,1.522488,18235.0,0.029951,0.659144,1,2548,1160,Hyperglycemia
1,210217-2-11,1.504415,18017.0,0.029593,0.652600,1,2548,1160,Hyperglycemia
2,210217-2-05,1.471457,17589.0,0.028890,0.640493,1,2548,1160,Hyperglycemia
3,210217-2-04,1.471432,17591.0,0.028893,0.640519,1,2548,1160,Hyperglycemia
4,210217-2-18,1.469541,17574.0,0.028865,0.639833,1,2548,1160,Hyperglycemia
...,...,...,...,...,...,...,...,...,...
3040,210519-2-37,0.417426,3510.0,0.005765,0.248813,1,2548,1160,Hyperglycemia
3041,210114-1-17,0.399481,3243.0,0.005327,0.241883,1,1,2299,Normal
3042,210304-2-20,0.397664,3213.0,0.005277,0.241136,1,1,2299,Hypoglycemia
3043,210211-2-27,0.386780,3080.0,0.005059,0.237206,1,1,2299,Hyperglycemia


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = graph_df.drop(columns=['SpecID', 'Status', 'Louvain', 'Leiden', 'LabelPropagation'])

# Target variable
y = graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)


Random Forest Accuracy: 0.38095238095238093
Extra Trees Accuracy: 0.3825944170771757


In [35]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [36]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [37]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.38095238095238093


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.37      0.28      0.32       200
 Hypoglycemia       0.39      0.45      0.41       198
       Normal       0.38      0.41      0.40       211

     accuracy                           0.38       609
    macro avg       0.38      0.38      0.38       609
 weighted avg       0.38      0.38      0.38       609


Confusion Matrix:
[[57 72 71]
 [42 89 67]
 [55 70 86]]


In [38]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.3825944170771757


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.36      0.28      0.32       200
 Hypoglycemia       0.39      0.47      0.43       198
       Normal       0.39      0.40      0.39       211

     accuracy                           0.38       609
    macro avg       0.38      0.38      0.38       609
 weighted avg       0.38      0.38      0.38       609


Confusion Matrix:
[[56 74 70]
 [42 93 63]
 [56 71 84]]


In [48]:
get_feature_importances(rf_model, X)

,Feature,Importance
0,PageRank,0.275189
3,ArticleRank,0.264383
2,EigenvectorCentrality,0.231722
1,DegreeCentrality,0.228705


In [49]:
get_feature_importances(et_model, X)

,Feature,Importance
0,PageRank,0.268130
3,ArticleRank,0.259141
2,EigenvectorCentrality,0.237516
1,DegreeCentrality,0.235212
